In [1]:
!nvidia-smi

Wed Jul 28 14:21:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## install

In [2]:
!pip install -q --upgrade pip
!pip install -q transformers pythainlp==2.2.4 tensorflow==2.4.0 tensorflow_text emoji seqeval sentencepiece fuzzywuzzy datasets
!npx degit --force https://github.com/vistec-AI/thai2transformers#dev

     |████████████████████████████████| 1.6 MB 4.3 MB/s 
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 2.6 MB 4.2 MB/s 
     |████████████████████████████████| 8.9 MB 14.4 MB/s 
     |████████████████████████████████| 394.7 MB 18 kB/s 
     |████████████████████████████████| 4.3 MB 56.7 MB/s 
     |████████████████████████████████| 185 kB 71.8 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 34.8 MB/s 
     |████████████████████████████████| 542 kB 44.6 MB/s 
     |████████████████████████████████| 743 kB 44.6 MB/s 
     |████

## load data

In [3]:
import pandas as pd
import os
import re
import numpy as np
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW
import torch
from tqdm import tqdm
import glob
import shutil
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

download AI for Thai (Thai QA wiki)

In [4]:
!gdown --id 14BNqgCOPJdQzwjJWPZGbqKeYu-9MHC1r
# https://drive.google.com/file/d/14BNqgCOPJdQzwjJWPZGbqKeYu-9MHC1r/view?usp=sharing
!unzip -q /content/AIFORTHAI-ThaiWikiQA_NSC2020.zip

!unzip -q /content/AIFORTHAI-ThaiWikiQA_NSC2020/wiki-documents-nsc.zip

Downloading...
From: https://drive.google.com/uc?id=14BNqgCOPJdQzwjJWPZGbqKeYu-9MHC1r
To: /content/AIFORTHAI-ThaiWikiQA_NSC2020.zip
170MB [00:00, 182MB/s]


download kaggle data

https://www.kaggle.com/c/ai-hackathon-online-2021-qa

In [5]:
!gdown --id 1iMOWolTN-MH9jzGsVGUnAUXGDu_TV7yh
# https://drive.google.com/file/d/1iMOWolTN-MH9jzGsVGUnAUXGDu_TV7yh/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1iMOWolTN-MH9jzGsVGUnAUXGDu_TV7yh
To: /content/ai-hackathon-online-2021-qa.zip
100% 488k/488k [00:00<00:00, 65.8MB/s]


In [6]:
!unzip -q /content/ai-hackathon-online-2021-qa.zip

copy test SCG data folder to wiki data folder

In [ ]:
# # for stupid bug: file name have no SCG
# def add_SCG_name(path):
#     for f in os.listdir(path):
#         os.rename(os.path.join(path, f), os.path.join(path, f))

# add_SCG_name('/content/test')

In [7]:
scg_file = glob.glob('/content/test/*.txt')
for f in scg_file:
    shutil.copy(f, '/content/wiki-documents-nsc')

In [15]:
def train_article_path(df, text_path=''):
    df['article_id'] = df['article_path']
    df['article_path'] = [os.path.join(text_path, x+'.txt') for x in df['article_id'].values]
    return df

scg_train = fix_bug_filename(pd.read_csv('/content/SCG-Train.csv', encoding="utf-8"), '/content/train')

scg_train.head()

,article_path,question,answer,article_id
0,/content/train/SCG102.txt,ตัวแทนจาก WEDO Developer “Team 26” ชื่ออะไร?,“Team 26”,SCG102
1,/content/train/SCG102.txt,มีกี่ประเทศในเอเชียตะวันออกเฉียงใต้ที่แข่งขันใ...,10,SCG102
2,/content/train/SCG102.txt,การแข่งขันมีชื่อว่าอะไร,AWS Microservices GameDay 2020,SCG102
3,/content/train/SCG102.txt,แข่งขันเกี่ยวกับอะไร,แข่งกันตั้งระบบจากโจทย์ที่ได้รับมาให้เสถียรและ...,SCG102
4,/content/train/SCG102.txt,ใครได้รับชัยชนะในการแข่งขันนี้,ตัวแทนจาก WEDO Developer “Team 26”,SCG102


In [19]:
# kaggle test
def test_article_path(df, test_text_path):
    df['article_path'] = [os.path.join(test_text_path, x+'.txt') for x in df['article_id']]
    return df

test = pd.read_csv('/content/Test.csv', encoding="utf-8")
test_path = '/content/wiki-documents-nsc'
test = test_article_path(test, test_path)
test.head()

,id,article_id,question,Unnamed: 3,article_path
0,1,220726,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt
1,2,8217,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt
2,3,35303,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt
3,4,12493,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt
4,5,11209,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt


In [21]:
# for wiki QA dataset
wiki_train = pd.read_csv('/content/AIFORTHAI-ThaiWikiQA_NSC2020/ThaiQACorpus-DevelopmentDataset.csv')
wiki_train = wiki_train[wiki_train['question_type']==1] # only type 1 question (คำถามอัตนัย)

wiki_text_path = '/content/wiki-documents-nsc'
wiki_train['article_path'] = [os.path.join(wiki_text_path, str(x)+'.txt') for x in wiki_train['article_id']]
wiki_train['article_name'] = wiki_train['article_id']
wiki_train['article_id'] = wiki_train.index
wiki_train.head()

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id,article_path,article_name
0,1,1,สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,ฮิกกิ้นส์,529.0,538.0,0,/content/wiki-documents-nsc/115035.txt,115035
1,2,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,เมชตา,139.0,144.0,1,/content/wiki-documents-nsc/376583.txt,376583
2,3,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,ลมสุริยะ,641.0,649.0,2,/content/wiki-documents-nsc/376583.txt,376583
3,4,1,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอ...,ปี 1933,304.0,311.0,3,/content/wiki-documents-nsc/547560.txt,547560
4,5,1,ห้องทำงานรูปไข่ สร้างขึ้นในสมัยอดีตประธานาธิบด...,ประธานาธิบดีวิลเลียม เอช. ทัฟต์,231.0,262.0,4,/content/wiki-documents-nsc/193032.txt,193032


In [22]:
# def readfiles(file_path):
#     with open(file_path,encoding="utf-8") as f:
#         art = f.read()
#     return clean_text(art)

# Preprocessing

In [23]:
def clean_text(text): # clean emoji
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def remove_HTML_syntax(input_text): # for wiki data
    text = ''
    accumulate = False
    output_text = input_text
    for char in input_text:
        if char == '<':
            accumulate = True
        elif char == '>':
            output_text = output_text.replace(text + char, '')
            text = ''
            accumulate = False
        
        if accumulate:
            text += char

    return output_text

def context_and_hashtag(file_path): # for hashtag question
    context_and_hashtag = {}
    with open(file_path, encoding="utf-8") as f:
        lines = f.readlines()
        hashtag = ""
        raw = ""
        for line in lines:
            if not line.startswith("#") and len(line.strip()) >= 2:
                line = clean_text(line)
                # line = line.replace("”", "").replace("“", "")
                line = line.strip()
                raw += line
            elif line.startswith("#"):
                 hashtag += line.strip()
    context_and_hashtag[file_path] = {"context": raw, "hashtag": hashtag}
    return context_and_hashtag
    
def preprocessing(df):
    df['context_and_hashtag'] = [context_and_hashtag(x) for x in df["article_path"]]
    df["context"] = [list(x.values())[0]['context'] for x in df['context_and_hashtag']]
    df["hashtag"] = [list(x.values())[0]['hashtag'] for x in df['context_and_hashtag']]
    df["Predicted"] = ""
    df['context'] = df['context'].map(lambda x: remove_HTML_syntax(x))
    return df

In [24]:
train_text_path = '/content/train'
test_text_path = '/content/wiki-documents-nsc'


scg_train = preprocessing(scg_train)
test = preprocessing(test)
wiki_train = preprocessing(wiki_train)

In [25]:
display(scg_train.head())
display(test.head())
display(wiki_train.head())

,article_path,question,answer,article_id,context_and_hashtag,context,hashtag,Predicted
0,/content/train/SCG102.txt,ตัวแทนจาก WEDO Developer “Team 26” ชื่ออะไร?,“Team 26”,SCG102,{'/content/train/SCG102.txt': {'context': 'ยิ่...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,#WEDO #Developer #AWS #AWSGameDay2020,
1,/content/train/SCG102.txt,มีกี่ประเทศในเอเชียตะวันออกเฉียงใต้ที่แข่งขันใ...,10,SCG102,{'/content/train/SCG102.txt': {'context': 'ยิ่...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,#WEDO #Developer #AWS #AWSGameDay2020,
2,/content/train/SCG102.txt,การแข่งขันมีชื่อว่าอะไร,AWS Microservices GameDay 2020,SCG102,{'/content/train/SCG102.txt': {'context': 'ยิ่...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,#WEDO #Developer #AWS #AWSGameDay2020,
3,/content/train/SCG102.txt,แข่งขันเกี่ยวกับอะไร,แข่งกันตั้งระบบจากโจทย์ที่ได้รับมาให้เสถียรและ...,SCG102,{'/content/train/SCG102.txt': {'context': 'ยิ่...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,#WEDO #Developer #AWS #AWSGameDay2020,
4,/content/train/SCG102.txt,ใครได้รับชัยชนะในการแข่งขันนี้,ตัวแทนจาก WEDO Developer “Team 26”,SCG102,{'/content/train/SCG102.txt': {'context': 'ยิ่...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,#WEDO #Developer #AWS #AWSGameDay2020,


,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted
0,1,220726,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,,
1,2,8217,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,,
2,3,35303,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,,
3,4,12493,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,,
4,5,11209,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,,


,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id,article_path,article_name,context_and_hashtag,context,hashtag,Predicted
0,1,1,สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,ฮิกกิ้นส์,529.0,538.0,0,/content/wiki-documents-nsc/115035.txt,115035,{'/content/wiki-documents-nsc/115035.txt': {'c...,เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู...,,
1,2,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,เมชตา,139.0,144.0,1,/content/wiki-documents-nsc/376583.txt,376583,{'/content/wiki-documents-nsc/376583.txt': {'c...,ลูนา 1 ลูนา 1 (อี-1 ซีรีส์) ซึ่งในขณะนั้นรู้จั...,,
2,3,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,ลมสุริยะ,641.0,649.0,2,/content/wiki-documents-nsc/376583.txt,376583,{'/content/wiki-documents-nsc/376583.txt': {'c...,ลูนา 1 ลูนา 1 (อี-1 ซีรีส์) ซึ่งในขณะนั้นรู้จั...,,
3,4,1,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอ...,ปี 1933,304.0,311.0,3,/content/wiki-documents-nsc/547560.txt,547560,{'/content/wiki-documents-nsc/547560.txt': {'c...,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อ กระทรวงโฆ...,,
4,5,1,ห้องทำงานรูปไข่ สร้างขึ้นในสมัยอดีตประธานาธิบด...,ประธานาธิบดีวิลเลียม เอช. ทัฟต์,231.0,262.0,4,/content/wiki-documents-nsc/193032.txt,193032,{'/content/wiki-documents-nsc/193032.txt': {'c...,ห้องทำงานรูปไข่ ห้องทำงานรูปไข่ (Oval Office) ...,,


# format data for fine-tuning

In [26]:
def find_start_end(df):
    answer_starts = []
    answer_stops = []
    for idx, row in df.iterrows():
        try:
            answer_start = row["context"].index(row["answer"])
            answer_stop = len(row['answer']) + answer_start
        except:
            answer_start = -1
            answer_stop = -1
        answer_starts.append(answer_start)
        answer_stops.append(answer_stop)

    return answer_starts, answer_stops

In [27]:
starts, stops = find_start_end(scg_train)
scg_train['answer_start'] = starts
scg_train['answer_stop'] = stops
scg_train['article_id'] = [str(x)+'scg' for x in scg_train.index]
scg_train.head()

,article_path,question,answer,article_id,context_and_hashtag,context,hashtag,Predicted,answer_start,answer_stop
0,/content/train/SCG102.txt,ตัวแทนจาก WEDO Developer “Team 26” ชื่ออะไร?,“Team 26”,0scg,{'/content/train/SCG102.txt': {'context': 'ยิ่...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,#WEDO #Developer #AWS #AWSGameDay2020,,57,66
1,/content/train/SCG102.txt,มีกี่ประเทศในเอเชียตะวันออกเฉียงใต้ที่แข่งขันใ...,10,1scg,{'/content/train/SCG102.txt': {'context': 'ยิ่...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,#WEDO #Developer #AWS #AWSGameDay2020,,257,259
2,/content/train/SCG102.txt,การแข่งขันมีชื่อว่าอะไร,AWS Microservices GameDay 2020,2scg,{'/content/train/SCG102.txt': {'context': 'ยิ่...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,#WEDO #Developer #AWS #AWSGameDay2020,,89,119
3,/content/train/SCG102.txt,แข่งขันเกี่ยวกับอะไร,แข่งกันตั้งระบบจากโจทย์ที่ได้รับมาให้เสถียรและ...,3scg,{'/content/train/SCG102.txt': {'context': 'ยิ่...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,#WEDO #Developer #AWS #AWSGameDay2020,,296,361
4,/content/train/SCG102.txt,ใครได้รับชัยชนะในการแข่งขันนี้,ตัวแทนจาก WEDO Developer “Team 26”,4scg,{'/content/train/SCG102.txt': {'context': 'ยิ่...,ยิ่ง challenge ยิ่งได้เรียนรู้! ตัวแทนจาก WEDO...,#WEDO #Developer #AWS #AWSGameDay2020,,32,66


In [28]:
scg_train.drop(scg_train[scg_train['answer_start']==-1].index, inplace=True)
len(scg_train)

269

In [29]:
wiki_train.drop(['question_type'], axis=1, inplace=True)
wiki_train.rename(columns={'answer_begin_position': 'answer_start', 'answer_end_position': 'answer_stop'}, inplace=True)
wiki_train['question_id'] = [str(x) + 'wiki' for x in wiki_train['question_id']]
wiki_train.head()

,question_id,question,answer,answer_start,answer_stop,article_id,article_path,article_name,context_and_hashtag,context,hashtag,Predicted
0,1wiki,สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,ฮิกกิ้นส์,529.0,538.0,0,/content/wiki-documents-nsc/115035.txt,115035,{'/content/wiki-documents-nsc/115035.txt': {'c...,เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู...,,
1,2wiki,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,เมชตา,139.0,144.0,1,/content/wiki-documents-nsc/376583.txt,376583,{'/content/wiki-documents-nsc/376583.txt': {'c...,ลูนา 1 ลูนา 1 (อี-1 ซีรีส์) ซึ่งในขณะนั้นรู้จั...,,
2,3wiki,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,ลมสุริยะ,641.0,649.0,2,/content/wiki-documents-nsc/376583.txt,376583,{'/content/wiki-documents-nsc/376583.txt': {'c...,ลูนา 1 ลูนา 1 (อี-1 ซีรีส์) ซึ่งในขณะนั้นรู้จั...,,
3,4wiki,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอ...,ปี 1933,304.0,311.0,3,/content/wiki-documents-nsc/547560.txt,547560,{'/content/wiki-documents-nsc/547560.txt': {'c...,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อ กระทรวงโฆ...,,
4,5wiki,ห้องทำงานรูปไข่ สร้างขึ้นในสมัยอดีตประธานาธิบด...,ประธานาธิบดีวิลเลียม เอช. ทัฟต์,231.0,262.0,4,/content/wiki-documents-nsc/193032.txt,193032,{'/content/wiki-documents-nsc/193032.txt': {'c...,ห้องทำงานรูปไข่ ห้องทำงานรูปไข่ (Oval Office) ...,,


In [30]:
wiki_and_scg_df = pd.concat([scg_train, wiki_train])
# wiki_and_scg_df.drop(['article_name'], axis=1, inplace=True)
wiki_and_scg_df.sample(5)

,article_path,question,answer,article_id,context_and_hashtag,context,hashtag,Predicted,answer_start,answer_stop,question_id,article_name
90,/content/wiki-documents-nsc/640889.txt,คิม ยอน-คยอง นักกีฬาวอลเลย์บอลหญิงทีมชาติเกาหล...,โรงเรียนมัธยมปลายสตรีวิทยาการคอมพิวเตอร์ซูว็อน,90,{'/content/wiki-documents-nsc/640889.txt': {'c...,โรงเรียนมัธยมปลายสตรีวิทยาการคอมพิวเตอร์ซูว็อน...,,,169.0,215.0,91wiki,640889.0
4476,/content/wiki-documents-nsc/159533.txt,สีเหลืองที่ปรากฏในธงชาติมอริเตเนียที่เริ่มใช้ใ...,ทรายในทะเลทรายซาฮารา,4476,{'/content/wiki-documents-nsc/159533.txt': {'c...,ธงชาติมอริเตเนีย ธงชาติมอริเตเนีย เป็นธงสี่เหล...,,,465.0,485.0,4477wiki,159533.0
11643,/content/wiki-documents-nsc/337175.txt,ไทลาซีนหรือที่รู้จักกันทั่วไปในชื่อว่าอะไร,หมาป่าแทสมาเนีย,11643,{'/content/wiki-documents-nsc/337175.txt': {'c...,"ไทลาซีน ไทลาซีน (; ; มาจากภาษากรีก แปลว่า ""มีห...",,,213.0,228.0,11644wiki,337175.0
3967,/content/wiki-documents-nsc/865806.txt,ลัทธิเหมาคือทฤษฎีทางการเมืองที่พัฒนามาจากคำสอน...,เหมา เจ๋อตง,3967,{'/content/wiki-documents-nsc/865806.txt': {'c...,ลัทธิเหมา ลัทธิเหมา คือ ทฤษฎีทางการเมืองที่พัฒ...,,,157.0,168.0,3968wiki,865806.0
835,/content/wiki-documents-nsc/233396.txt,หม่อมดุษฎี บริพัตร ณ อยุธยา ได้ถึงแก่อนิจกรรมด...,โรงพยาบาลศิริราช,835,{'/content/wiki-documents-nsc/233396.txt': {'c...,หม่อมดุษฎี บริพัตร ณ อยุธยา หม่อมดุษฎี บริพัตร...,,,2026.0,2042.0,836wiki,233396.0


In [31]:
def format_training(df):
    format_list_all = []
    for row in df.itertuples():
        format_dict = {'answers': {'answer_end': [row.answer_stop], 
                                   'answer_start': [row.answer_start],
                                   'text': [row.answer]},
                       'article_id': row.article_id,
                       'context': row.context,
                       'question': row.question,
                       'question_id': str(row.article_id),
                       'title': row.article_name
                       }
        format_list_all.append(format_dict)
    return format_list_all

# scg_data_train = format_training(scg_train)
wiki_and_scg_data = format_training(wiki_and_scg_df)

In [32]:
test['article_name'] = test['article_id']
test['article_id'] = test.index
test['question_id'] = [str(x)+'test' for x in test['article_id']]
test.head()

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,,,220726,0test
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,,,8217,1test
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,,,35303,2test
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,,,12493,3test
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,,,11209,4test


In [33]:
def format_test(df):
    format_list_all = []
    for row in df.itertuples():
        format_dict = {'article_id': row.article_id,
                       'context': row.context,
                       'question': row.question,
                       'question_id': str(str(row.article_id)+'test'),
                       'title': row.article_name
                       }
        format_list_all.append(format_dict)
    return format_list_all

test_data = format_test(test)

# 1. คำถาม อัตนัย

แบ่งเป็นทำ Short context model กับ Long context model

โดยให้ถ้า Short context model ไม่ตอบ ให้ Long context model ตอยแทน

## for Long context

Disclaimer: code ส่วนใหญ่มาจาก 
https://colab.research.google.com/github/vistec-AI/thai2transformers/blob/dev/notebooks/train_question_answering_lm_finetuning.ipynb?fbclid=IwAR1bxgNTVbc402CT46xzhEfftCgkoIZi76Zcxxtv63E6jUaaLCZWPQTLhxM#scrollTo=XYt9LrfLgTUj

จากทีมทำ WangchanBERTa

In [34]:
import collections
import logging
import pprint
import re
from tqdm.auto import tqdm

import numpy as np
import torch

#datasets
from datasets import (
    load_dataset, 
    load_metric, 
    concatenate_datasets,
    load_from_disk,
)

#transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
)

#thai2transformers
import thai2transformers
from thai2transformers.metrics import (
    squad_newmm_metric,
    question_answering_metrics,
)
from thai2transformers.preprocess import (
    prepare_qa_train_features
)
from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer,
    ThaiWordsNewmmTokenizer,
    ThaiWordsSyllableTokenizer,
    FakeSefrCutTokenizer,
    SEFR_SPLIT_TOKEN
)

### choose pre-trained model

ในการใช้งาน ส่วนตัวผมใช้ deepset/xlm-roberta-large-squad2

In [36]:
model_names = [
    'wangchanberta-base-att-spm-uncased',
    'xlm-roberta-base',
    'bert-base-multilingual-cased',
    'wangchanberta-base-wiki-newmm',
    'wangchanberta-base-wiki-ssg',
    'wangchanberta-base-wiki-sefr',
    'wangchanberta-base-wiki-spm',
    'deepset/xlm-roberta-large-squad2',
    'mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp'

]

tokenizers = {
    'wangchanberta-base-att-spm-uncased': AutoTokenizer,
    'xlm-roberta-base': AutoTokenizer,
    'bert-base-multilingual-cased': AutoTokenizer,
    'wangchanberta-base-wiki-newmm': ThaiWordsNewmmTokenizer,
    'wangchanberta-base-wiki-ssg': ThaiWordsSyllableTokenizer,
    'wangchanberta-base-wiki-sefr': FakeSefrCutTokenizer,
    'wangchanberta-base-wiki-spm': ThaiRobertaTokenizer,
    'deepset/xlm-roberta-large-squad2':AutoTokenizer,
    'mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp':AutoTokenizer
}

public_models = ['xlm-roberta-base', 'bert-base-multilingual-cased', 'deepset/xlm-roberta-large-squad2', 'monsoon-nlp/bert-base-thai', 'mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp'] 
model_name = "deepset/xlm-roberta-large-squad2" #@param ["wangchanberta-base-att-spm-uncased", "xlm-roberta-base", "bert-base-multilingual-cased", "wangchanberta-base-wiki-newmm", "wangchanberta-base-wiki-syllable", "wangchanberta-base-wiki-sefr", "wangchanberta-base-wiki-spm", "deepset/xlm-roberta-large-squad2", "deepset/xlm-roberta-large-squad2", "mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp"]


In [37]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = tokenizers[model_name].from_pretrained(
                f'airesearch/{model_name}' if model_name not in public_models else f'{model_name}',
                revision='main',
                model_max_length=416,)

Downloading:   0%|          | 0.00/606 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/179 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

XLMRobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
          

In [39]:
#parameterizing columns
CONTEXT_COL = 'context'
QUESTION_COL = 'question'
ANSWERS_COL = 'answers'
TEXT_COL = 'text'
START_COL = 'answer_start'
END_COL = 'answer_end'
QUESTION_ID_COL = 'question_id'

In [40]:
# train-terst split
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(wiki_and_scg_data, test_size=0.05, random_state=42)
print(len(train_data), len(val_data))

14505 764


สร้างข้อมูลในรูป Datasets

In [41]:
 from datasets import Dataset

 def create_datasets(data):
    t_dict = {'answers': [],
                            'article_id': [],
                            'context': [],
                            'question': [],
                            'question_id': [],
                            'title': []
                            }

    for d in data:
        t_dict['answers'].append(d['answers'])
        t_dict['article_id'].append(str(d['article_id']))
        t_dict['context'].append(d['context'])
        t_dict['question'].append(d['question'])
        t_dict['question_id'].append(d['question_id'])
        t_dict['title'].append(str(d['title']))
    
    datasets = Dataset.from_dict(t_dict)

    return datasets

def create_test_dataset(data):
    test_dict = {'article_id': [],
            'context': [],
            'question': [],
            'question_id': [],
            'title': []
            }
    for d in data:
        test_dict['article_id'].append(d['article_id'])
        test_dict['context'].append(d['context'])
        test_dict['question'].append(d['question'])
        test_dict['question_id'].append(d['question_id'])
        test_dict['title'].append(str(d['title']))
    
    datasets = Dataset.from_dict(test_dict)
    return datasets

In [42]:
datasets_train = create_datasets(train_data)
datasets_val = create_datasets(val_data)

In [43]:
datasets_test = create_test_dataset(test_data)
datasets_test

Dataset({
    features: ['article_id', 'context', 'question', 'question_id', 'title'],
    num_rows: 5608
})

In [44]:
max_length = 416 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [45]:
def _get_context_span(input_ids,
                     sequence_ids,
                     answers, 
                     start_col, 
                     text_col='text',
                     pad_on_right=True):
    
    # Start/end character index of the answer in the text.
    start_char = answers[start_col][0] 
    end_char = start_char + len(answers[text_col][0]) + 1

    # Start token index of the current span in the text.
    token_start_index = 0
    while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
        token_start_index += 1

    # End token index of the current span in the text.
    token_end_index = len(input_ids) - 1
    while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
        token_end_index -= 1
    
    return token_start_index, token_end_index, start_char, end_char

def _get_answer_span(tokenized_examples,
                    offsets,
                    start_char,
                    end_char,
                    token_start_index,
                    token_end_index,):

    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1

    return start_position, end_position

def prepare_qa_train_features(examples, 
                           tokenizer,
                           question_col='question',
                           context_col='context',
                           answers_col='answers',
                           start_col='answer_start',
                           text_col='text',
                           pad_on_right=True,
                           max_length=416,
                           doc_stride=128):
    
    tokenized_examples = tokenizer(
        examples[question_col if pad_on_right else context_col],
        examples[context_col if pad_on_right else question_col],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride, #overlapping of overflowing tokens
        return_overflowing_tokens=True, #return multiple input ids if exceeding max_length
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping") #map overflowing examples to original examples
    offset_mapping = tokenized_examples.pop("offset_mapping") #offset map; character index not resetted across overflowing examples

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    #loop through all examples' offset_mapping
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i] #a list of tokens
        cls_index = input_ids.index(tokenizer.cls_token_id) #use cls as answer when there is no answer 
        sequence_ids = tokenized_examples.sequence_ids(i) #sequence_ids 0 for question and 1 for context
        sample_index = sample_mapping[i] #since many examples can point to the same question due to overflowing
        answers = examples[answers_col][sample_index] #answers of each example

        # If no answers are given, set the cls_index as answer
        if len(answers[text_col]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            token_start_index, token_end_index, start_char, end_char = _get_context_span(input_ids=input_ids,
                                                                sequence_ids=sequence_ids,
                                                                answers=answers, 
                                                                start_col=start_col, 
                                                                text_col=text_col,
                                                                pad_on_right=pad_on_right)
            # If answer is not in span, return cls_index as answer
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:                
                start_position, end_position = _get_answer_span(tokenized_examples,
                    offsets,
                    start_char,
                    end_char,
                    token_start_index,
                    token_end_index,)
                tokenized_examples["start_positions"].append(start_position)
                tokenized_examples["end_positions"].append(end_position)
                
    return tokenized_examples

In [47]:
#using function in notebook
tokenized_datasets_train = datasets_train.map(lambda x: prepare_qa_train_features(x, tokenizer), 
                                  batched=True, 
                                  remove_columns=datasets_train.column_names)

tokenized_datasets_val = datasets_val.map(lambda x: prepare_qa_train_features(x, tokenizer), 
                                  batched=True, 
                                  remove_columns=datasets_val.column_names)

tokenized_datasets_train

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions'],
    num_rows: 58741
})

### Finetune model

In [ ]:
batch_size = 4
learning_rate = 2e-5

args = TrainingArguments(
    f"finetune_SCG_QA_xlm_roberta",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size*2,
    num_train_epochs=1,
    warmup_ratio=0.2,
    weight_decay=0.01,
    fp16=True,
    save_total_limit=3,
    # load_best_model_at_end=True,
)

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
# trainer.save_model("/content/drive/MyDrive/finetune_SCG_QA_xlm_roberta")

### infer

In [59]:
# #load finetuned model
# model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/finetune_SCG_QA_xlm_roberta')

# trainer = Trainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets_train,
#     eval_dataset=tokenized_datasets_val,
#     data_collator=data_collator,
#     tokenizer=tokenizer,
# )

In [48]:
args = TrainingArguments(
    f"xlm_roberta_base",
    evaluation_strategy = "epoch",
    # learning_rate=learning_rate,
    # per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size*2,
    num_train_epochs=2,
    warmup_ratio=0.2,
    weight_decay=0.01,
    fp16=True,
    save_total_limit=3,
    # load_best_model_at_end=True,
)
data_collator = default_data_collator
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using amp fp16 backend


In [49]:
def prepare_validation_features(examples, 
                           tokenizer,
                           question_col='question',
                           context_col='context',
                           question_id_col = 'question_id',
                           pad_on_right=True,
                           max_length=416,
                           doc_stride=128):

    tokenized_examples = tokenizer(
        examples[question_col if pad_on_right else context_col],
        examples[context_col if pad_on_right else question_col],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []
    pass

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples[question_id_col][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [50]:
def _postprocess_qa_predictions(examples,
                               features, 
                               raw_predictions,
                               tokenizer,
                               n_best_size = 20, 
                               max_answer_length = 30,
                               allow_no_answer=False,
                               question_id_col='question_id'):
    
    #get start_logits and end_logits
    all_start_logits, all_end_logits = raw_predictions

    #get `offset_mapping` and `example_id` back
    features.set_format(type=features.format["type"], columns=list(features.features.keys()))

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples[question_id_col])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]
        context = example["context"]

        min_null_score = None
        valid_answers = []
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            # This is what will allow us to map some the positions in our logits to span of texts in the original context.
            offset_mapping = features[feature_index]["offset_mapping"]

            #debug
            input_ids = features[feature_index]['input_ids'] 

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index-1] is None #end_index is the exclusive upperbound
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index-1][1] #end_index is the exclusive upperbound
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text_decode": tokenizer.decode(input_ids[start_index:end_index]),
                            "text": context[start_char:end_char],
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid failure.
            best_answer = {"text": "", "score": 0.0}
            
        if not allow_no_answer:
            predictions[example[question_id_col]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example[question_id_col]] = answer

    return predictions

In [51]:
#features
test_features = datasets_test.map(
    lambda x: prepare_validation_features(x, tokenizer),
    batched=True,
    remove_columns=datasets_test.column_names
)

#raw predictions
raw_predictions = trainer.predict(test_features)

  0%|          | 0/6 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 19841
  Batch size = 8


In [52]:
question_id_predictions = _postprocess_qa_predictions(datasets_test, test_features, raw_predictions.predictions, tokenizer)

  0%|          | 0/5608 [00:00<?, ?it/s]

In [72]:
# XLM roberta
results = question_id_predictions
ts = []
rs = [] 
for i in results.items():
    ts.append(i[0])
    rs.append(i[1])

xlm_results = pd.DataFrame({'question_id':ts, 'prediction':rs})
# xlm_results = xlm_results.replace(r'^\s*$', np.nan, regex=True)
xlm_results[:20]

,question_id,prediction
0,0test,วิเชียร ตันติพิมล
1,1test,แมวเปอร์เซียและแมวขน
2,2test,จตุรัง
3,3test,Hemidacty
4,4test,เสียงคำรามของเสือนั้นจะดังก้องกังวานไปทั่ว
5,5test,ประธานาธิบดีแห่งสาธารณรัฐ
6,6test,เอดัวร์ ฟี
7,7test,กัมพูชา
8,8test,เทือน ประสิทธิ์ และเอียง ซา
9,9test,ไท-กะ


#### เพิ่ม BERT 
(ใช้ในกรณีที่ ถ้าคำถามไหน xlm_roberta ไม่ตอบคำถามนั้น ให้ใช้คำตอบจาก BERT แทนที่จะไม่ตอบ)

In [56]:
model_name_bert = 'mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp'

In [57]:
model_bert = AutoModelForQuestionAnswering.from_pretrained(model_name_bert)
tokenizer_bert = tokenizers[model_name_bert].from_pretrained(
                f'airesearch/{model_name_bert}' if model_name_bert not in public_models else f'{model_name_bert}',
                revision='main',
                model_max_length=416,)

https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7fkieb25


Downloading:   0%|          | 0.00/657 [00:00<?, ?B/s]

storing https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/66a1a2fcf63972a264c297e96b41c58d31fa6cfc46fb3d9f266fc0bbadc297e9.b94e26d9c4c6febfe5881b469240758616a103ec32f181f3bec6660712555f9b
creating metadata file for /root/.cache/huggingface/transformers/66a1a2fcf63972a264c297e96b41c58d31fa6cfc46fb3d9f266fc0bbadc297e9.b94e26d9c4c6febfe5881b469240758616a103ec32f181f3bec6660712555f9b
loading configuration file https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/66a1a2fcf63972a264c297e96b41c58d31fa6cfc46fb3d9f266fc0bbadc297e9.b94e26d9c4c6febfe5881b469240758616a103ec32f181f3bec6660712555f9b
Model config BertConfig {
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "

Downloading:   0%|          | 0.00/711M [00:00<?, ?B/s]

storing https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/b4f01d30e396164cd666137ae73c9c0bdd8efc24e3d3f1848ef1feb16d1da406.7f28f06c145df4bf7b5b80325cbc44a294440a61166daab86a2eaac87dfc227a
creating metadata file for /root/.cache/huggingface/transformers/b4f01d30e396164cd666137ae73c9c0bdd8efc24e3d3f1848ef1feb16d1da406.7f28f06c145df4bf7b5b80325cbc44a294440a61166daab86a2eaac87dfc227a
loading weights file https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/b4f01d30e396164cd666137ae73c9c0bdd8efc24e3d3f1848ef1feb16d1da406.7f28f06c145df4bf7b5b80325cbc44a294440a61166daab86a2eaac87dfc227a
All model checkpoint weights were used when initializing BertForQuestionAnswering.

All the weights of BertForQuestionAnswering were initialized from the model checkpoint at mrm8488/bert-multi-cas

...

storing https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/9bf945a50ba5f04e49a9b599037071acd150812ae971444f7f508e799af8ba8c.3e813ef7b46a58d7e35c90b0aacc2127a3800bba6bff791af1651012cf2fab11
creating metadata file for /root/.cache/huggingface/transformers/9bf945a50ba5f04e49a9b599037071acd150812ae971444f7f508e799af8ba8c.3e813ef7b46a58d7e35c90b0aacc2127a3800bba6bff791af1651012cf2fab11
loading configuration file https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/66a1a2fcf63972a264c297e96b41c58d31fa6cfc46fb3d9f266fc0bbadc297e9.b94e26d9c4c6febfe5881b469240758616a103ec32f181f3bec6660712555f9b
Model config BertConfig {
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidd

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

storing https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/6ea3c660695d3ae107ec7bcf05dd06940fc642773e8c8c2f85e952064f42c127.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
creating metadata file for /root/.cache/huggingface/transformers/6ea3c660695d3ae107ec7bcf05dd06940fc642773e8c8c2f85e952064f42c127.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpvxvnrck9


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/e0c8fa9208ce0a8ea5e0df356afec8a97e65a2230299501022a24f0b8e99c517.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/e0c8fa9208ce0a8ea5e0df356afec8a97e65a2230299501022a24f0b8e99c517.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6ea3c660695d3ae107ec7bcf05dd06940fc642773e8c8c2f85e952064f42c127.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/mrm8488/bert-multi-cased-finedtu

In [58]:
args = TrainingArguments(
    f"bert-multi-cased-finedtuned-xquad-tydiqa-goldp",
    evaluation_strategy = "epoch",
    # learning_rate=learning_rate,
    # per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size*2,
    num_train_epochs=2,
    warmup_ratio=0.2,
    weight_decay=0.01,
    fp16=True,
    save_total_limit=3,
    # load_best_model_at_end=True,
)
data_collator = default_data_collator
trainer = Trainer(
    model_bert,
    args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    data_collator=data_collator,
    tokenizer=tokenizer_bert,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend


In [59]:
#features
test_features = datasets_test.map(
    lambda x: prepare_validation_features(x, tokenizer_bert),
    batched=True,
    remove_columns=datasets_test.column_names
)

#raw predictions
raw_predictions_bert = trainer.predict(test_features)

  0%|          | 0/6 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 44262
  Batch size = 8


In [61]:
question_id_predictions_bert = _postprocess_qa_predictions(datasets_test, test_features, raw_predictions_bert.predictions, tokenizer_bert)

  0%|          | 0/5608 [00:00<?, ?it/s]

In [73]:
## BERT
results_bert = question_id_predictions_bert

ts = []
rs = [] 
for i in results_bert.items():
    ts.append(i[0])
    rs.append(i[1])

bert_results = pd.DataFrame({'question_id':ts, 'prediction':rs})
# bert_results = bert_results.replace(r'^\s*$', np.nan, regex=True)
bert_results[:20]

,question_id,prediction
0,0test,สุระชัย ถวิลไพ
1,1test,แมวอะบิสซิเนีย และแมวโซมา
2,2test,จตุรังก
3,3test,Hemida
4,4test,แกนกระดูกไฮออย
5,5test,ประธานาธิบดีแห่งสาธารณรัฐฝ
6,6test,เอดัวร์ ฟีลิ
7,7test,กัมพูชาประชาธิปไต
8,8test,พล พตและเอียง ซาร
9,9test,ไท-กะไ


## for short context
แม่นกว่าใช้อันที่ Long context model มาทำshort context

### XLM-RoBERTa

In [139]:
pretrained = "deepset/xlm-roberta-large-squad2" #@param ["wangchanberta-base-att-spm-uncased", "xlm-roberta-base", "bert-base-multilingual-cased", "wangchanberta-base-wiki-newmm", "wangchanberta-base-wiki-syllable", "wangchanberta-base-wiki-sefr", "wangchanberta-base-wiki-spm", "deepset/xlm-roberta-large-squad2", "monsoon-nlp/bert-base-thai"]


In [140]:
name = pretrained

In [141]:
tokenizer = AutoTokenizer.from_pretrained(name, revision='main')
# tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForQuestionAnswering.from_pretrained(name)

loading configuration file https://huggingface.co/deepset/xlm-roberta-large-squad2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/531c1582e1ea0b7d34c7de10efd3593838f1018f8d012b8029c9283c41cba7c0.09d513aaf4fbccf6b8b4d0264d74ea7dc8d6fb056bdb099e45621b06d8c877de
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "name": "XLMRoberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.1",
  "type_vocab_size": 1,
  "use_cache": true,
  

In [142]:
model.to('cuda')
print(1)

1


In [143]:
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [145]:
def find_answer(question, context, tokenizer=tokenizer, model=model):
    inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt", truncation=True, max_length=512).to('cuda')

    if inputs['attention_mask'].size()[1] > 512:
        inputs['input_ids'] = inputs['input_ids'][:, :512]
        inputs['attention_mask'] = inputs['attention_mask'][:, :512]
       
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    try:
        # answer_start_scores, answer_end_scores = model(**inputs)
        model_ans = model(**inputs)
        answer_start_scores = model_ans.start_logits
        answer_end_scores = model_ans.end_logits
        
        answer_start = torch.argmax(answer_start_scores)  
        answer_score = torch.max(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1
        
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
        if answer.strip():
            return answer,  answer_score
        else:
            return "<UNK/>", -1
    except Exception as e:
        print(e)
        return "<UNK/>", -1

def main_test(df, t, m):
    for row in tqdm(df.itertuples()):
        question = row.question.strip()
        context = row.context.strip()
        hashtag = row.hashtag.strip()
        lines = context.split("<SEP />")

        #hashtag answer
        hashtag = hashtag.replace("\n", " ")
        question = question.replace("?", "")
        question_hashtag = " ".join([h for h in question.split(" ") if h.startswith("#")])
        answer_hashtag = " ".join([h for h in hashtag.split(" ") if h.startswith("#")])

        # hashtag answer
        if "#" in question or "แฮชแทก" in question or "แฮชแท็ก" in question or "hashtag" in question.lower():
            if question_hashtag:
                for h in question_hashtag.split(" "):
                    answer_hashtag = answer_hashtag.replace(h, "")
            predict = answer_hashtag.strip()
            df.loc[row.Index, "Predicted_hashtag"] = predict.strip()
        else:
            predict, score = find_answer(question, context, tokenizer=t, model=m)
        
            df.loc[row.Index, "Predicted"] = predict.strip()
    return df

In [146]:
xlm_roberta_ = main_test(test, tokenizer, model)

5608it [05:55, 15.78it/s]


### Include BERT

In [148]:
pretrained = "mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp" #@param ["mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp", "wangchanberta-base-att-spm-uncased", "xlm-roberta-base", "bert-base-multilingual-cased", "wangchanberta-base-wiki-newmm", "wangchanberta-base-wiki-syllable", "wangchanberta-base-wiki-sefr", "wangchanberta-base-wiki-spm", "deepset/xlm-roberta-large-squad2", "monsoon-nlp/bert-base-thai"]


In [149]:
name = pretrained

In [150]:
tokenizer_bert = AutoTokenizer.from_pretrained(name, use_fast=False)
# tokenizer = AutoTokenizer.from_pretrained(name)
model_bert = AutoModelForQuestionAnswering.from_pretrained(name)

loading configuration file https://huggingface.co/mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/66a1a2fcf63972a264c297e96b41c58d31fa6cfc46fb3d9f266fc0bbadc297e9.b94e26d9c4c6febfe5881b469240758616a103ec32f181f3bec6660712555f9b
Model config BertConfig {
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding

In [151]:
model_bert.to('cuda')

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [ ]:
bert_short = main_test(test, tokenizer_bert, model_bert)

In [157]:
bert_short.head()

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,Predicted_yesno
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,,ประยุทธ์ จันทร์โอชา,220726,0test,NaN,สุระชัย ถวิลไพ,NaN
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,,[CLS],8217,1test,NaN,แมวอะบิสซิเนีย และแมวโซมา,NaN
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,,จตุรังกา,35303,2test,NaN,จตุรังก,NaN
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,,Hemidactylus,12493,3test,NaN,Hemida,NaN
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,,[CLS] อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่...,11209,4test,NaN,แกนกระดูกไฮออย,NaN


In [158]:
display(xlm_roberta_short.head())
display(bert_short.head())

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,Predicted_yesno
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,,<s>,220726,0test,NaN,สุระชัย ถวิลไพ,NaN
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,,<s>,8217,1test,NaN,แมวอะบิสซิเนีย และแมวโซมา,NaN
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,,จตุรังกา,35303,2test,NaN,จตุรังก,NaN
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,,Hemidactylus,12493,3test,NaN,Hemida,NaN
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,,หากินเวลากลางคืน มีถิ่นกําเนิดในป่า เสือส่วนให...,11209,4test,NaN,แกนกระดูกไฮออย,NaN


,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,Predicted_yesno
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,,ประยุทธ์ จันทร์โอชา,220726,0test,NaN,สุระชัย ถวิลไพ,NaN
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,,[CLS],8217,1test,NaN,แมวอะบิสซิเนีย และแมวโซมา,NaN
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,,จตุรังกา,35303,2test,NaN,จตุรังก,NaN
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,,Hemidactylus,12493,3test,NaN,Hemida,NaN
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,,[CLS] อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่...,11209,4test,NaN,แกนกระดูกไฮออย,NaN


In [ ]:
# evaluation



## อันนี้ยังเขียนไม่เสร็จ และคงจะไม่เสร็จ dealing with long sequence (not works)

In [ ]:
def new_find_answer(question, context, tokenizer=tokenizer, model=model, device='cuda'):
    inputs = tokenizer(question, 
                        context,
                        max_length=max_length,
                        truncation="only_second",
                        return_overflowing_tokens=True,
                        stride=doc_stride,
                    )
    
    # partition context (with overlapping)

    number_context = len(inputs['input_ids'])

    # print('number_context: ', number_context)
    iter = 0 # loob for all sub cotexts

    compare_score_subcontext = {'answer':[], 'score':[]}
    while iter <= number_context-1:
        input_ids = inputs["input_ids"][iter]
        attention_mask = inputs['attention_mask'][iter]
        text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

        input_ids = torch.LongTensor(input_ids) # list to tensor
        input_ids = torch.unsqueeze(input_ids, 0).to(device).detach() # shape ===> (length) -> (1, lenght)
        attention_mask = torch.FloatTensor(attention_mask) # list to tensor
        attention_mask = torch.unsqueeze(attention_mask, 0).to(device).detach()


        # XLM-roberta parameters dosen't have return_overflowing_tokens, offset_mapping and stride
        # we use these parameters to make sub contexts
        # https://huggingface.co/transformers/model_doc/xlmroberta.html#xlmrobertaforquestionanswering

        model_inputs = {'input_ids':input_ids, 'attention_mask':attention_mask}
        model_ans = model(input_ids=model_inputs['input_ids'], attention_mask=model_inputs['attention_mask'])

        answer_start_scores = model_ans.start_logits
        answer_end_scores = model_ans.end_logits

        answer_start = torch.argmax(answer_start_scores)  
        answer_end = torch.argmax(answer_end_scores) + 1
        answer_score = torch.max(answer_start_scores)

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))

        if answer.strip():
            compare_score_subcontext['answer'].append(answer)
            compare_score_subcontext['score'].append(answer_score)
        else:
            compare_score_subcontext['answer'].append("<UNK/>")
            compare_score_subcontext['score'].append(torch.LongTensor([0]))
        iter += 1

    if len(compare_score_subcontext['answer']) > 1:
        ## compare score sub context 
        ## if answer [<s>, <UNK/>, 'asdfasfsa']
        ## we use 'asdfasfsa' whatever answer_score
        ## if answer ['123', '456'], we wil consider the score
        ans = compare_score_subcontext.get('answer')
        score_tensor = compare_score_subcontext.get('score')
        score = [x.item() for x in score_tensor]
        s_ans = []
        unk_answer = []

        for a in ans:
            # check <s>, <UNK/>
            s_ans.append(check_s_UNK_token_index(a, '<s>'))
            unk_answer.append(check_s_UNK_token_index(a, '<UNK/>'))

        s_ans_index = np.argmax(np.array(s_ans))
        unk_answer_index = np.argmax(np.array(unk_answer))

        # replace <s>, <UNK/> answer score with -1000
        score[s_ans_index] = -1000
        score[unk_answer_index] = -1000
    else:
        ans = compare_score_subcontext.get('answer')
        score_tensor = compare_score_subcontext.get('score')
        score = [x.item() for x in score_tensor]

    # argmax score 
    ans_index = np.argmax(np.array(score))
    final_answer = ans[ans_index]
    final_score = score[ans_index]

    torch.cuda.empty_cache()
    gc.collect()
    return final_answer, final_score

In [ ]:
def check_s_UNK_token_index(ans, s_or_unk=''):
    try:
        index = ans.index(s_or_unk)
        index = 1
    except:
        index = -1 # -1 means there is no <s> or <UNK/> tokens in answer
    return index

def new_main_test(df, t, m):
    for row in tqdm(df.itertuples()):
        question = row.question.strip()
        context = row.context.strip()
        hashtag = row.hashtag.strip()
        lines = context.split("<SEP />")

        #hashtag answer
        hashtag = hashtag.replace("\n", " ")
        question = question.replace("?", "")
        question_hashtag = " ".join([h for h in question.split(" ") if h.startswith("#")])
        answer_hashtag = " ".join([h for h in hashtag.split(" ") if h.startswith("#")])

        # hashtag answer
        if "#" in question or "แฮชแทก" in question or "แฮชแท็ก" in question or "hashtag" in question.lower():
            if question_hashtag:
                for h in question_hashtag.split(" "):
                    answer_hashtag = answer_hashtag.replace(h, "")
            predict = answer_hashtag.strip()
        else:
            predict, score = new_find_answer(question, context, tokenizer=t, model=m)
        
        df.loc[row.Index, "Predicted"] = predict.strip()

    return df

In [ ]:
xlm_long = test.copy()
xlm_long.drop(xlm_long.index, inplace=True)

In [ ]:
xlm_long.head()

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted


In [ ]:
for _, test_group in test.groupby('article_id'):
    xlm_long_group = new_main_test(test_group, tokenizer, model)
    xlm_long = pd.concat([xlm_long, xlm_long_group])
    
display(xlm_long)

1it [00:02,  2.08s/it]
2it [00:25, 12.56s/it]
1it [00:11, 11.66s/it]

In [ ]:
new_main_test(test, tokenizer, model)

In [ ]:
xlm_roberta_longsequenceable = new_main_test(test, tokenizer, model)

In [ ]:
xlm_roberta_long = xlm_roberta_longsequenceable.copy()
xlm_roberta_long.head()

In [63]:
def find_answer(question, context, tokenizer=tokenizer, model=model):
    inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt", truncation=True, max_length=512).to('cuda')

    if inputs['attention_mask'].size()[1] > 512:
        inputs['input_ids'] = inputs['input_ids'][:, :512]
        inputs['attention_mask'] = inputs['attention_mask'][:, :512]
       
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    try:
        # answer_start_scores, answer_end_scores = model(**inputs)
        model_ans = model(**inputs)
        answer_start_scores = model_ans.start_logits
        answer_end_scores = model_ans.end_logits
        
        answer_start = torch.argmax(answer_start_scores)  
        answer_score = torch.max(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1
        
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
        if answer.strip():
            return answer,  answer_score
        else:
            return "<UNK/>", -1
    except Exception as e:
        print(e)
        return "<UNK/>", -1

def main_test(df, t, m):
    for row in tqdm(df.itertuples()):
        question = row.question.strip()
        context = row.context.strip()
        hashtag = row.hashtag.strip()
        lines = context.split("<SEP />")

        #hashtag answer
        hashtag = hashtag.replace("\n", " ")
        question = question.replace("?", "")
        question_hashtag = " ".join([h for h in question.split(" ") if h.startswith("#")])
        answer_hashtag = " ".join([h for h in hashtag.split(" ") if h.startswith("#")])

        # hashtag answer
        if "#" in question or "แฮชแทก" in question or "แฮชแท็ก" in question or "hashtag" in question.lower():
            if question_hashtag:
                for h in question_hashtag.split(" "):
                    answer_hashtag = answer_hashtag.replace(h, "")
            predict = answer_hashtag.strip()
            df.loc[row.Index, "Predicted_hashtag"] = predict.strip()
        # else:
        #     predict, score = find_answer(question, context, tokenizer=t, model=m)
        
        #     df.loc[row.Index, "Predicted"] = predict.strip()
    return df

In [72]:
xlm_roberta_ = main_test(test, tokenizer, model)

0it [00:00, ?it/s]

In [73]:
xlm_roberta = xlm_roberta_.copy()
xlm_roberta

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,,<s>,220726,0test,NaN
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,,<s>,8217,1test,NaN
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,,จตุรังกา,35303,2test,NaN
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,,Hemidactylus,12493,3test,NaN
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,,หากินเวลากลางคืน มีถิ่นกําเนิดในป่า เสือส่วนให...,11209,4test,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5603,5604,5603,จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,,3 เดือน 20 วัน,SCG217,5603test,NaN
5604,5605,5604,เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,,User Experience design (UX),SCG217,5604test,NaN
5605,5606,5605,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,,<s>,SCG217,5605test,NaN
5606,5607,5606,ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,,keep update อยู่เสมอ,SCG217,5606test,NaN


In [ ]:
xlm_roberta.to_csv('only_xlm_fulltest.csv')

In [147]:
xlm_roberta_short = xlm_roberta_.copy()
xlm_roberta_short

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,Predicted_yesno
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,,<s>,220726,0test,NaN,สุระชัย ถวิลไพ,NaN
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,,<s>,8217,1test,NaN,แมวอะบิสซิเนีย และแมวโซมา,NaN
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,,จตุรังกา,35303,2test,NaN,จตุรังก,NaN
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,,Hemidactylus,12493,3test,NaN,Hemida,NaN
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,,หากินเวลากลางคืน มีถิ่นกําเนิดในป่า เสือส่วนให...,11209,4test,NaN,แกนกระดูกไฮออย,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5603,5604,5603,จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,,3 เดือน 20 วัน,SCG217,5603test,NaN,3 เดือน,NaN
5604,5605,5604,เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,,User Experience design (UX),SCG217,5604test,NaN,User Experience,NaN
5605,5606,5605,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,,<s>,SCG217,5605test,NaN,Digitally Transforming,NaN
5606,5607,5606,ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,,keep update อยู่เสมอ,SCG217,5606test,NaN,Dis,NaN


# 2. Yes/no question

ใช้ deepset/xlm-roberta-large-squad2 model

In [92]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

In [93]:
# Use a GPU if you have one available (Runtime -> Change runtime type -> GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds for reproducibility
random.seed(26)
np.random.seed(26)
torch.manual_seed(26)

In [94]:
model_name = "deepset/xlm-roberta-large-squad2" #@param ["wangchanberta-base-att-spm-uncased", "xlm-roberta-base", "bert-base-multilingual-cased", "wangchanberta-base-wiki-newmm", "wangchanberta-base-wiki-syllable", "wangchanberta-base-wiki-sefr", "wangchanberta-base-wiki-spm", "deepset/xlm-roberta-large-squad2", "monsoon-nlp/bert-base-thai", "roberta-base"]


In [95]:
torch.cuda.empty_cache() 

In [96]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device) # Send the model to the GPU if we have one
tokenizer = AutoTokenizer.from_pretrained(model_name) 

loading configuration file https://huggingface.co/deepset/xlm-roberta-large-squad2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/531c1582e1ea0b7d34c7de10efd3593838f1018f8d012b8029c9283c41cba7c0.09d513aaf4fbccf6b8b4d0264d74ea7dc8d6fb056bdb099e45621b06d8c877de
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "name": "XLMRoberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.1",
  "type_vocab_size": 1,
  "use_cache": true,
  

In [97]:
def encode_data(tokenizer, questions, contexts, max_length):
    """Encode the question/context pairs into features than can be fed to the model."""
    
    input_ids = []
    attention_masks = []

    for question, context in zip(questions, contexts):
        encoded_data = tokenizer.encode_plus(question, context, max_length=max_length, pad_to_max_length=True, truncation_strategy="longest_first")
        encoded_pair = encoded_data["input_ids"]
        attention_mask = encoded_data["attention_mask"]

        input_ids.append(encoded_pair)
        attention_masks.append(attention_mask)

    return np.array(input_ids), np.array(attention_masks)

In [98]:
# train test split
yes_no_df = pd.read_csv('/content/AIFORTHAI-ThaiWikiQA_NSC2020/ThaiQACorpus-DevelopmentDataset.csv')
yes_no_df = yes_no_df[yes_no_df['question_type']==2]
yes_no_df.head()

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id
15000,15001,2,ภาษาฮีบรูอาซเกนาซีเป็นระบบการออกเสียงของภาษาฮี...,ไม่ใช่,NaN,NaN,265515
15001,15002,2,แองคิโลซอรัสเป็นไดโนเสาร์ที่เล็กที่สุดในสกุล แ...,ไม่ใช่,NaN,NaN,319289
15002,15003,2,ลูกตุ้มขนาดใหญ่ของ แองคิโลซอรัส เป็นส่วนกระดูก...,ไม่ใช่,NaN,NaN,319289
15003,15004,2,แอบถ่าย หรือโรคถ้ำมองเป็นความสนใจหรือข้อประพฤต...,ไม่ใช่,NaN,NaN,773798
15004,15005,2,เอเชียนอินดอร์-มาร์เชียลอาร์ตสเกมส์เป็นการแข่ง...,ไม่ใช่,NaN,NaN,358523


In [99]:
text_yesno_path = '/content/wiki-documents-nsc'

yes_no_df['article_path']  = [os.path.join(text_yesno_path, str(x)+'.txt') for x in yes_no_df['article_id']]

booleanDictionary = {'ใช่': True, 'ไม่ใช่': False}
yes_no_df['answer'] = yes_no_df['answer'].replace(booleanDictionary)

yes_no_df = preprocessing(yes_no_df)[['question_id', 'question', 'answer', 'article_id', 'article_path', 'context']]
yes_no_df.head()

,question_id,question,answer,article_id,article_path,context
15000,15001,ภาษาฮีบรูอาซเกนาซีเป็นระบบการออกเสียงของภาษาฮี...,False,265515,/content/wiki-documents-nsc/265515.txt,ภาษาฮีบรูอาซเกนาซี ภาษาฮีบรูอาซเกนาซี (Ashkena...
15001,15002,แองคิโลซอรัสเป็นไดโนเสาร์ที่เล็กที่สุดในสกุล แ...,False,319289,/content/wiki-documents-nsc/319289.txt,แองคิโลซอรัส แองคิโลซอรัส () เป็นไดโนเสาร์ที่ใ...
15002,15003,ลูกตุ้มขนาดใหญ่ของ แองคิโลซอรัส เป็นส่วนกระดูก...,False,319289,/content/wiki-documents-nsc/319289.txt,แองคิโลซอรัส แองคิโลซอรัส () เป็นไดโนเสาร์ที่ใ...
15003,15004,แอบถ่าย หรือโรคถ้ำมองเป็นความสนใจหรือข้อประพฤต...,False,773798,/content/wiki-documents-nsc/773798.txt,โรคถ้ำมอง แอบถายโรคถ้ำมอง () หรือ ความชอบแอบดู...
15004,15005,เอเชียนอินดอร์-มาร์เชียลอาร์ตสเกมส์เป็นการแข่ง...,False,358523,/content/wiki-documents-nsc/358523.txt,เอเชียนอินดอร์-มาร์เชียลอาร์ตสเกมส์ เอเชียนอิน...


In [100]:
train_yes_no, test_yes_no = train_test_split(yes_no_df, test_size=0.33, random_state=42)
display(train_yes_no.head())
display(test_yes_no.head())

,question_id,question,answer,article_id,article_path,context
15081,15082,นายแพทย์กันตพงศ์ เล่าลือพงศ์ศิริ จบการศึกษาเมื...,False,169093,/content/wiki-documents-nsc/169093.txt,กันตพงศ์ เล่าลือพงศ์ศิริ นายแพทย์กันตพงศ์ เล่า...
15915,15916,มูนไลต์ เป็นรายการโทรทัศน์จากสหรัฐอเมริกา ที่ส...,True,182098,/content/wiki-documents-nsc/182098.txt,มูนไลต์ มูนไลต์ () เป็นรายการโทรทัศน์จากสหรัฐอ...
16018,16019,มาซีโดเนียวิทยุและโทรทัศน์เป็นทรัพย์สินของทางส...,True,256700,/content/wiki-documents-nsc/256700.txt,สถานีวิทยุโทรทัศน์แห่งชาติมาซิโดเนีย สถานีวิทย...
15380,15381,จานบิน หรือจานผี เป็นคำทั่วไปใช้เรียกวัตถุบินท...,False,120401,/content/wiki-documents-nsc/120401.txt,จานบิน จานบิน () หรือจานผี เป็นวัตถุบิน ลักษณะ...
16029,16030,ถ้าความดันไอถูกผสมด้วยก๊าซอื่นเรียก ความดันย่อ...,True,23799,/content/wiki-documents-nsc/23799.txt,ความดันไอ ความดันไอ () คือ ความดัน (ถ้าไอถูกผส...


,question_id,question,answer,article_id,article_path,context
16860,16861,บีกานามัยซินถูกดูดซึมได้น้อยมากในทางเดินอาหาร ...,False,911752,/content/wiki-documents-nsc/911752.txt,บีกานามัยซิน บีกานามัยซิน หรือ กานามัยซิน บี (...
15353,15354,"""ยูบีลองวิทมี""เป็นเพลงคันทรีป็อปของนักร้องนักแ...",False,313746,/content/wiki-documents-nsc/313746.txt,"ยูบีลองวิทมี ""ยูบีลองวิทมี"" () เป็นเพลงแนวคันท..."
16333,16334,ลินดา หลิน ไต้เป็นนางเอกภาพยนตร์ภาษาจีนกลางในส...,True,423334,/content/wiki-documents-nsc/423334.txt,หลิน ไต้ ลินดา หลิน ไต้ (; ) เป็นนักแสดงหญิงชา...
15905,15906,ทรูคริปต์เป็นชุดคำสั่งที่ถูกสร้างขึ้นเพื่อใช้ใ...,True,151119,/content/wiki-documents-nsc/151119.txt,ทรูคริปต์ ทรูคริปต์ () เป็นซอฟต์แวร์ประเภทโอเพ...
16289,16290,มาตลีเทพบุตรถือของหอมและดอกไม้ทิพย์โปรยปรายในร...,True,58298,/content/wiki-documents-nsc/58298.txt,ปางเสด็จลงจากดาวดึงส์ ปางเสด็จลงจากดาวดึงส์ ทำ...


## Train

In [ ]:
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)

In [ ]:
contexts_train = train_yes_no.context.values
questions_train = train_yes_no.question.values
answers_train = train_yes_no.answer.values.astype(int)

In [ ]:
contexts_test = test_yes_no.context.values
questions_test = test_yes_no.question.values
answers_test = test_yes_no.answer.values.astype(int)

In [ ]:
# Encoding data
max_seq_length = 412
input_ids_train, attention_masks_train = encode_data(tokenizer, questions_train, contexts_train, max_seq_length)
input_ids_test, attention_masks_test = encode_data(tokenizer, questions_test, contexts_test, max_seq_length)

train_features = (input_ids_train, attention_masks_train, answers_train)
test_features = (input_ids_test, attention_masks_test, answers_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
batch_size = 2

train_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in train_features]
test_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in test_features]

train_dataset = TensorDataset(*train_features_tensors)
test_dataset = TensorDataset(*test_features_tensors)

train_sampler = RandomSampler(train_dataset)
dev_sampler = SequentialSampler(test_dataset)

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, sampler=dev_sampler, batch_size=batch_size)

In [ ]:
epochs = 2
grad_acc_steps = 1
display_steps = 1
train_loss_values = []
test_acc_values = []

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
for epoch in range(epochs):
    # Training
    epoch_train_loss = 0 # Cumulative loss
    model.train()
    model.zero_grad()

    for step, batch in tqdm(enumerate(train_dataloader), desc='Steps: '):

        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2].to(device) 

        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)

        loss = outputs[0]
        loss = loss / grad_acc_steps
        epoch_train_loss += loss.item()

        loss.backward()

        if (step+1) % grad_acc_steps == 0: # Gradient accumulation is over
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clipping gradients
            optimizer.step()
            model.zero_grad()
        # torch.cuda.empty_cache() 

    epoch_train_loss = epoch_train_loss / len(train_dataloader)          
    train_loss_values.append(epoch_train_loss)

    if epoch % display_steps == 0:
        print('Train: ', epoch_train_loss)

    # Evaluation
    epoch_dev_accuracy = 0 # Cumulative accuracy
    model.eval()

    for batch in test_dataloader:
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2]
                
        with torch.no_grad():        
            outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
                    
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()

        predictions = np.argmax(logits, axis=1).flatten()
        labels = labels.numpy().flatten()

        epoch_dev_accuracy += np.sum(predictions == labels) / len(labels)

    epoch_dev_accuracy = epoch_dev_accuracy / len(test_dataloader)
    test_acc_values.append(epoch_dev_accuracy)

    if epoch % display_steps == 0:
        print('Test: ', epoch_dev_accuracy)

In [ ]:
torch.save(model, '/content/drive/MyDrive/QA_yesno_model.pth')

## Inferrence

In [110]:
# load trained model ทำครั้งแรกต้อง train เองนะ
model = torch.load('/content/drive/MyDrive/QA_yesno_model.pth')
model.to('cuda')

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
     

In [111]:
def predict(question, context, max_length=416):
    sequence = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=max_length, truncation_strategy="longest_first")['input_ids'].to(device)
    logits = model(sequence)[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    # proba_yes = round(probabilities[1], 2)
    # proba_no = round(probabilities[0], 2)
    prediction = np.array(probabilities).argmax()
    # print(f"Question: {question}, Yes: {proba_yes}, No: {proba_no}")
    return int(prediction) # 1->Yes, 0-> NO

In [112]:
def run_yes_no(df):
    yes_no_prediction = []
    for row in tqdm(df.itertuples()):
        question = str(row.question)
        context = str(row.context)

        if "ใช่หรือไม่" in question or "หรือไม่" in question or question in yes_no_df['question'].values:
            yes_no_prediction.append(predict(question, context))
        else:
            yes_no_prediction.append(np.nan)
    df['yes_no_prediction'] = yes_no_prediction
    return df

#### evaluation

In [ ]:
test_yes_no['id'] = test_yes_no['question_id']

In [ ]:
pred_yes_no = run_yes_no(test_yes_no)

In [115]:
pred_yes_no

,question_id,question,answer,article_id,article_path,context,id,yes_no_prediction
16860,16861,บีกานามัยซินถูกดูดซึมได้น้อยมากในทางเดินอาหาร ...,0,911752,/content/wiki-documents-nsc/911752.txt,บีกานามัยซิน บีกานามัยซิน หรือ กานามัยซิน บี (...,16861,0
15353,15354,"""ยูบีลองวิทมี""เป็นเพลงคันทรีป็อปของนักร้องนักแ...",0,313746,/content/wiki-documents-nsc/313746.txt,"ยูบีลองวิทมี ""ยูบีลองวิทมี"" () เป็นเพลงแนวคันท...",15354,0
16333,16334,ลินดา หลิน ไต้เป็นนางเอกภาพยนตร์ภาษาจีนกลางในส...,1,423334,/content/wiki-documents-nsc/423334.txt,หลิน ไต้ ลินดา หลิน ไต้ (; ) เป็นนักแสดงหญิงชา...,16334,1
15905,15906,ทรูคริปต์เป็นชุดคำสั่งที่ถูกสร้างขึ้นเพื่อใช้ใ...,1,151119,/content/wiki-documents-nsc/151119.txt,ทรูคริปต์ ทรูคริปต์ () เป็นซอฟต์แวร์ประเภทโอเพ...,15906,1
16289,16290,มาตลีเทพบุตรถือของหอมและดอกไม้ทิพย์โปรยปรายในร...,1,58298,/content/wiki-documents-nsc/58298.txt,ปางเสด็จลงจากดาวดึงส์ ปางเสด็จลงจากดาวดึงส์ ทำ...,16290,1
...,...,...,...,...,...,...,...,...
15118,15119,ไอดี เร็คคอร์ด (ID Records) เป็นค่ายเพลงในเครื...,0,306029,/content/wiki-documents-nsc/306029.txt,ไอดี เร็คคอร์ด ไอดี เร็คคอร์ด (ID Records) เป็...,15119,0
16249,16250,คนที่เป็นปากแหว่งเพดานโหว่ ถ้าหากรอยแยกต่อเนื่...,1,225458,/content/wiki-documents-nsc/225458.txt,ปากแหว่งเพดานโหว่ ปากแหว่ง (; ) และเพดานโหว่ (...,16250,1
16993,16994,เลขอารบิก เป็นสัญลักษณ์ตัวเลขที่ใช้เพียงบางส่ว...,0,37489,/content/wiki-documents-nsc/37489.txt,ตัวเลขอารบิก เลขอารบิก เป็นสัญลักษณ์ตัวเลขที่น...,16994,0
15522,15523,วุฒิภาวะคือ การควบคุมอารมณ์ความต้องการของตนเอง...,0,247685,/content/wiki-documents-nsc/247685.txt,วุฒิภาวะ วุฒิภาวะ () คือ ความสามารถในการยับยั้...,15523,0


In [116]:
from sklearn.metrics import accuracy_score
pred_yes_no.dropna(inplace=True)
pred_yes_no['yes_no_prediction'] = pred_yes_no['yes_no_prediction'].astype(int)
pred_yes_no["answer"] = pred_yes_no["answer"].astype(int)

accuracy_score(pred_yes_no['answer'], pred_yes_no['yes_no_prediction'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

0.9318181818181818

#### using testset

In [118]:
t = test.copy()
t = run_yes_no(t)
t.head()

5608it [00:24, 225.43it/s]


,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,yes_no_prediction
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,,วิเชียร ตันติพิมล,220726,0test,NaN,สุระชัย ถวิลไพ,NaN
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,,แมวเปอร์เซียและแมวขน,8217,1test,NaN,แมวอะบิสซิเนีย และแมวโซมา,NaN
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,,จตุรัง,35303,2test,NaN,จตุรังก,NaN
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,,Hemidacty,12493,3test,NaN,Hemida,NaN
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,,เสียงคำรามของเสือนั้นจะดังก้องกังวานไปทั่ว,11209,4test,NaN,แกนกระดูกไฮออย,NaN


In [119]:
booleanDictionary = {1: 'ใช่', 0: 'ไม่ใช่'}
t['yes_no_prediction'] = t['yes_no_prediction'].replace(booleanDictionary)

In [120]:
t[t['yes_no_prediction'].notnull()]

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,yes_no_prediction
1072,1073,1072,ยาคุมกำเนิดประเภทไหนที่ทำให้เกิดฝ้าใบหน้า น้ำห...,NaN,/content/wiki-documents-nsc/168085.txt,{'/content/wiki-documents-nsc/168085.txt': {'c...,ยาคุมกำเนิดชนิดฉีด ยาฉีดคุมกำเนิด () เป็นวิธีค...,,Progestero,168085,1072test,NaN,ประจำเดือนมาไม่แน่น,ใช่
3660,3661,3660,ลักษณะของการเลือกที่จะพิมพ์หรือไม่พิมพ์ผลงานวิ...,NaN,/content/wiki-documents-nsc/710583.txt,{'/content/wiki-documents-nsc/710583.txt': {'c...,ความเอนเอียงโดยการรายงาน ในวิทยาการระบาด ความเ...,,publication bias,710583,3660test,NaN,ความเอนเอียงในการตีพิมพ,ใช่
5000,5001,5000,การรถไฟรัสเซียเป็นบริษัทรถไฟในประเทศเดนมาร์ก แ...,NaN,/content/wiki-documents-nsc/607652.txt,{'/content/wiki-documents-nsc/607652.txt': {'c...,การรถไฟรัสเซีย การรถไฟรัสเซีย (JSC RZD; Rossiy...,,เป็นบริษัทรถไฟในประเทศรัสเซีย เป็นหนึ่งในบริษ...,607652,5000test,NaN,การรถไฟรัสเซีย การรถไฟรัส,ไม่ใช่
5001,5002,5001,คาสิโอเปีย เป็นวงดนตรีร็อกญี่ปุ่นที่ก่อตั้งขึ้...,NaN,/content/wiki-documents-nsc/271399.txt,{'/content/wiki-documents-nsc/271399.txt': {'c...,Casiopea คาสิโอเปีย (; ) เป็นวงดนตรีแจ๊สฟิวชัน...,,แจ๊สฟิวชัน,271399,5001test,NaN,Casio,ไม่ใช่
5002,5003,5002,เซี่ยเหมินเป็นเมืองในมณฑลฝูเจี้ยน มีชื่อในภาษา...,NaN,/content/wiki-documents-nsc/92824.txt,{'/content/wiki-documents-nsc/92824.txt': {'co...,เซี่ยเหมิน เซี่ยเหมิน (; ) เป็นเมืองในมณฑลฝูเจ...,,(Ē-mn̂g,92824,5002test,NaN,Ē-,ใช่
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5495,5496,5495,กีฬาซีเกมส์ 1981 (พ.ศ. 2524) ใช่กีฬาที่จัดการแ...,NaN,/content/wiki-documents-nsc/60306.txt,{'/content/wiki-documents-nsc/60306.txt': {'co...,ซีเกมส์ 1981 กีฬาซีเกมส์ 1981 (พ.ศ. 2524) ถือไ...,,ถือได้ว่าเป็นการจัดการแข่งขันซีเกมส์ ครั้งที่,60306,5495test,NaN,ประเทศฟิลิปปิน,ใช่
5496,5497,5496,ซานเซบัสเตียนเป็นเมืองรองของจังหวัดกิปุซโกอาใน...,NaN,/content/wiki-documents-nsc/84807.txt,{'/content/wiki-documents-nsc/84807.txt': {'co...,ซานเซบัสเตียน ซานเซบัสเตียน () หรือ โดโนสเตีย ...,,โดโนสเต,84807,5496test,NaN,โดโนสเต,ไม่ใช่
5497,5498,5497,"ซานเซบัสเตียน มีประชากรประมาณ 82,930 คนใช่หรือไม่",NaN,/content/wiki-documents-nsc/84807.txt,{'/content/wiki-documents-nsc/84807.txt': {'co...,ซานเซบัสเตียน ซานเซบัสเตียน () หรือ โดโนสเตีย ...,,"182,9",84807,5497test,NaN,"182,",ไม่ใช่
5498,5499,5498,เซาท์เทิร์นครอสเกมส์ 1978 เป็นการแข่งขันกีฬาเซ...,NaN,/content/wiki-documents-nsc/916674.txt,{'/content/wiki-documents-nsc/916674.txt': {'c...,เซาท์เทิร์นครอสเกมส์ 1978 เซาท์อเมริกันเกมส์ 1...,,โบลิ,916674,5498test,NaN,ประเทศโบลิเว,ไม่ใช่


# Combine

รวมคำตอบแต่ละแบบ ในแต่ละคำถาม

**priority:** yes/no & hashtag -> xlm short -> xlm long -> bert short -> bert long

## current combine

In [160]:
combine = xlm_roberta_short
combine['Predicted_bert'] = bert_short['Predicted'] # bert short
combine['Predicted_xlm_long'] = xlm_results['prediction'] # xlmroberta long
combine['Predicted_bert_long'] = bert_results['prediction'] # bert long
combine['Predicted_yesno'] = t['yes_no_prediction'] # yes/no question

combine.head()

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,Predicted_yesno,Predicted_xlm_long,Predicted_bert_long
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,,<s>,220726,0test,NaN,ประยุทธ์ จันทร์โอชา,NaN,วิเชียร ตันติพิมล,สุระชัย ถวิลไพ
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,,<s>,8217,1test,NaN,[CLS],NaN,แมวเปอร์เซียและแมวขน,แมวอะบิสซิเนีย และแมวโซมา
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,,จตุรังกา,35303,2test,NaN,จตุรังกา,NaN,จตุรัง,จตุรังก
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,,Hemidactylus,12493,3test,NaN,Hemidactylus,NaN,Hemidacty,Hemida
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,,หากินเวลากลางคืน มีถิ่นกําเนิดในป่า เสือส่วนให...,11209,4test,NaN,[CLS] อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่...,NaN,เสียงคำรามของเสือนั้นจะดังก้องกังวานไปทั่ว,แกนกระดูกไฮออย


In [164]:
def del_token(row):
    special_tokens = ['[CLS]', '<unk>', '<UNK>','<UNK/>', '<s>', '[UNK]', '</s>', '[SEP]']
    for t in special_tokens:
        row.Predicted = str(row.Predicted).replace(t, "")
        row.Predicted_bert = str(row.Predicted_bert).replace(t, "")
    return row

combine = combine.apply(del_token, axis='columns')
combine = combine.replace(r'^\s*$', np.nan, regex=True)
combine.head()

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,Predicted_yesno,Predicted_xlm_long,Predicted_bert_long
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,NaN,NaN,220726,0test,NaN,ประยุทธ์ จันทร์โอชา,NaN,วิเชียร ตันติพิมล,สุระชัย ถวิลไพ
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,NaN,NaN,8217,1test,NaN,NaN,NaN,แมวเปอร์เซียและแมวขน,แมวอะบิสซิเนีย และแมวโซมา
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,NaN,จตุรังกา,35303,2test,NaN,จตุรังกา,NaN,จตุรัง,จตุรังก
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,NaN,Hemidactylus,12493,3test,NaN,Hemidactylus,NaN,Hemidacty,Hemida
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,NaN,หากินเวลากลางคืน มีถิ่นกําเนิดในป่า เสือส่วนให...,11209,4test,NaN,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่ส...,NaN,เสียงคำรามของเสือนั้นจะดังก้องกังวานไปทั่ว,แกนกระดูกไฮออย


In [165]:
# xlmroberta short ไม่ตอบ ให้ xlmroberta long ตอบ
combine['Predicted'] = combine['Predicted'].combine_first(combine['Predicted_xlm_long']).to_frame() 

In [166]:
# แก้คำ bert short/long
combine['Predicted_bert'] = combine['Predicted_bert'].replace({'#':''}, regex=True)
combine['Predicted_bert'] = combine['Predicted_bert'].replace({'พ . ศ . ':'พ.ศ.'}, regex=True)
combine['Predicted_bert_long'] = combine['Predicted_bert_long'].replace({'#':''}, regex=True)
combine['Predicted_bert_long'] = combine['Predicted_bert_long'].replace({'พ . ศ . ':'พ.ศ.'}, regex=True)

# xlmroberta ไม่ตอบ ให้ bert short ตอบ
combine['Predicted'] = combine['Predicted'].combine_first(combine['Predicted_bert']).to_frame() 

# xlmroberta ไม่ตอบ ให้ bert long ตอบ
combine['Predicted'] = combine['Predicted'].combine_first(combine['Predicted_bert_long']).to_frame() 

In [169]:
# yes/no 
combine['Predicted'] = combine['Predicted_yesno'].combine_first(combine['Predicted']).to_frame()
combine = combine.replace(r'^\s*$', np.nan, regex=True)

# hashtag
combine['Predicted'] = combine['Predicted_hashtag'].combine_first(combine['Predicted']).to_frame()
combine = combine.replace(r'^\s*$', np.nan, regex=True)

In [170]:
combine.head()

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,Predicted_yesno,Predicted_xlm_long,Predicted_bert_long
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,NaN,วิเชียร ตันติพิมล,220726,0test,NaN,ประยุทธ์ จันทร์โอชา,NaN,วิเชียร ตันติพิมล,สุระชัย ถวิลไพ
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,NaN,แมวเปอร์เซียและแมวขน,8217,1test,NaN,NaN,NaN,แมวเปอร์เซียและแมวขน,แมวอะบิสซิเนีย และแมวโซมา
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,NaN,จตุรังกา,35303,2test,NaN,จตุรังกา,NaN,จตุรัง,จตุรังก
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,NaN,Hemidactylus,12493,3test,NaN,Hemidactylus,NaN,Hemidacty,Hemida
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,NaN,หากินเวลากลางคืน มีถิ่นกําเนิดในป่า เสือส่วนให...,11209,4test,NaN,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่ส...,NaN,เสียงคำรามของเสือนั้นจะดังก้องกังวานไปทั่ว,แกนกระดูกไฮออย


# submission

In [97]:
# yes/no 
combine['yes_no_prediction'] = t['yes_no_prediction']
combine['Predicted'] = combine['yes_no_prediction'].combine_first(combine['Predicted']).to_frame()
combine = combine.replace(r'^\s*$', np.nan, regex=True)

# hashtag
combine['Predicted'] = combine['Predicted_hashtag'].combine_first(combine['Predicted']).to_frame()
combine = combine.replace(r'^\s*$', np.nan, regex=True)

In [98]:
# hashtag
combine['Predicted'] = combine['Predicted_hashtag'].combine_first(combine['Predicted']).to_frame()
combine = combine.replace(r'^\s*$', np.nan, regex=True)

In [99]:
combine

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,prediction_hardcore,yes_no_prediction
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,NaN,วิเชียร ตันติพิมล,220726,0test,NaN,สุระชัย ถวิลไพ,วิเชียร ตันติพิมล,NaN
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,NaN,แมวเปอร์เซียและแมวขน,8217,1test,NaN,แมวอะบิสซิเนีย และแมวโซมา,แมวเปอร์เซียและแมวขน,NaN
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,NaN,จตุรังกา,35303,2test,NaN,จตุรังก,จตุรัง,NaN
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,NaN,Hemidactylus,12493,3test,NaN,Hemida,Hemidacty,NaN
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,NaN,หากินเวลากลางคืน มีถิ่นกําเนิดในป่า เสือส่วนให...,11209,4test,NaN,แกนกระดูกไฮออย,เสียงคำรามของเสือนั้นจะดังก้องกังวานไปทั่ว,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5603,5604,5603,จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,NaN,3 เดือน 20 วัน,SCG217,5603test,NaN,3 เดือน,3 เดือน 20,NaN
5604,5605,5604,เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,NaN,User Experience design (UX),SCG217,5604test,NaN,User Experience,User Experience design (UX,NaN
5605,5606,5605,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,NaN,1. consulting firms เจ้าใหญ่และบริษัท technol...,SCG217,5605test,NaN,Digitally Transforming,1. consulting firms เจ้าใหญ่และบริษัท technol...,NaN
5606,5607,5606,ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,NaN,keep update อยู่เสมอ,SCG217,5606test,NaN,Dis,Learning By Do,NaN


In [171]:
combine[combine['Predicted'].isna()]

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,Predicted_yesno,Predicted_xlm_long,Predicted_bert_long
1715,1716,1715,พระที่นั่งอัมพรสถานเป็นพระที่นั่ง 3 ชั้น ผังอา...,NaN,/content/wiki-documents-nsc/20093.txt,{'/content/wiki-documents-nsc/20093.txt': {'co...,พระที่นั่งอัมพรสถาน พระที่นั่งอัมพรสถาน เป็นพร...,NaN,NaN,20093,1715test,NaN,NaN,NaN,NaN,NaN
2983,2984,2983,วอลเลซ เสต็กเนอร์ ได้รับรางวัลของสภาศิลปะแห่งแ...,NaN,/content/wiki-documents-nsc/282981.txt,{'/content/wiki-documents-nsc/282981.txt': {'c...,วอลเลซ เสต็กเนอร์ วอลเลซ เสต็กเนอร์ () (18 กุม...,NaN,NaN,282981,2983test,NaN,NaN,NaN,NaN,NaN
3357,3358,3357,ในปี ค.ศ. 1946 ไวท์ย้ายไปอยู่ที่อัลเดอร์นีย์ โ...,NaN,/content/wiki-documents-nsc/713107.txt,{'/content/wiki-documents-nsc/713107.txt': {'c...,ที. เอช. ไวท์ เทเรนซ์ แฮนเบอรี ไวท์ (; 29 พฤษภ...,NaN,NaN,713107,3357test,NaN,NaN,NaN,NaN,NaN
3496,3497,3496,ศาลฎีกามีแผนกคดีพิเศษทั้งสิ้นกี่แผนก,NaN,/content/wiki-documents-nsc/316119.txt,{'/content/wiki-documents-nsc/316119.txt': {'c...,ศาลฎีกา ศาลฎีกา เป็นศาลยุติธรรมชั้นสูงสุด มีเข...,NaN,NaN,316119,3496test,NaN,NaN,NaN,NaN,NaN
3575,3576,3575,การให้เหตุผลสิทธิมนุษยชนสมัยใหม่กำเนิดขึ้นในช่...,NaN,/content/wiki-documents-nsc/28602.txt,{'/content/wiki-documents-nsc/28602.txt': {'co...,สิทธิมนุษยชน สิทธิมนุษยชน () เป็นหลักทางศีลธรร...,NaN,NaN,28602,3575test,NaN,NaN,NaN,NaN,NaN
3996,3997,3996,ธานินทร์ กรัยวิเชียร นายกรัฐมนตรีคนที่ 14 ของป...,NaN,/content/wiki-documents-nsc/657931.txt,{'/content/wiki-documents-nsc/657931.txt': {'c...,คาเรน กรัยวิเชียร คุณหญิงคาเรน กรัยวิเชียร (สก...,NaN,NaN,657931,3996test,NaN,NaN,NaN,NaN,NaN


In [172]:
def del_token(row):
    special_tokens = ['[CLS]', '<unk>', '<UNK>','<UNK/>', '<s>', '[UNK]', '</s>', '[SEP]']
    for t in special_tokens:
        row.Predicted = str(row.Predicted).replace(t, "")
        row.Predicted = str(row.Predicted).strip()
        # row.Predicted_bert = str(row.Predicted_bert).replace(t, "")
    return row

In [173]:
submit = combine
submit = submit.apply(del_token, axis='columns')
submit

,id,article_id,question,Unnamed: 3,article_path,context_and_hashtag,context,hashtag,Predicted,article_name,question_id,Predicted_hashtag,Predicted_bert,Predicted_yesno,Predicted_xlm_long,Predicted_bert_long
0,1,0,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,/content/wiki-documents-nsc/220726.txt,{'/content/wiki-documents-nsc/220726.txt': {'c...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,NaN,วิเชียร ตันติพิมล,220726,0test,NaN,ประยุทธ์ จันทร์โอชา,NaN,วิเชียร ตันติพิมล,สุระชัย ถวิลไพ
1,2,1,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,/content/wiki-documents-nsc/8217.txt,{'/content/wiki-documents-nsc/8217.txt': {'con...,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,NaN,แมวเปอร์เซียและแมวขน,8217,1test,NaN,NaN,NaN,แมวเปอร์เซียและแมวขน,แมวอะบิสซิเนีย และแมวโซมา
2,3,2,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,/content/wiki-documents-nsc/35303.txt,{'/content/wiki-documents-nsc/35303.txt': {'co...,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,NaN,จตุรังกา,35303,2test,NaN,จตุรังกา,NaN,จตุรัง,จตุรังก
3,4,3,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,/content/wiki-documents-nsc/12493.txt,{'/content/wiki-documents-nsc/12493.txt': {'co...,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,NaN,Hemidactylus,12493,3test,NaN,Hemidactylus,NaN,Hemidacty,Hemida
4,5,4,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,/content/wiki-documents-nsc/11209.txt,{'/content/wiki-documents-nsc/11209.txt': {'co...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,NaN,หากินเวลากลางคืน มีถิ่นกําเนิดในป่า เสือส่วนให...,11209,4test,NaN,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่ส...,NaN,เสียงคำรามของเสือนั้นจะดังก้องกังวานไปทั่ว,แกนกระดูกไฮออย
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5603,5604,5603,จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,NaN,3 เดือน 20 วัน,SCG217,5603test,NaN,3 เดือน 20,NaN,3 เดือน 20,3 เดือน
5604,5605,5604,เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,NaN,User Experience design (UX),SCG217,5604test,NaN,NaN,NaN,User Experience design (UX,User Experience
5605,5606,5605,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,NaN,1. consulting firms เจ้าใหญ่และบริษัท technolo...,SCG217,5605test,NaN,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมก...,NaN,1. consulting firms เจ้าใหญ่และบริษัท technol...,Digitally Transforming
5606,5607,5606,ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,NaN,/content/wiki-documents-nsc/SCG217.txt,{'/content/wiki-documents-nsc/SCG217.txt': {'c...,﻿เมื่อมาทำงานจริงกับ Digitally Transforming Or...,NaN,keep update อยู่เสมอ,SCG217,5606test,NaN,NaN,NaN,Learning By Do,Dis


In [174]:
s = pd.read_csv("/content/SampleSubmission.csv", encoding='utf_8_sig')
s

,id,answer
0,1,วิเชียร ตันติพิมลพันธุ์
1,2,แมวเปอร์เซียและแมวขนสั้น
2,3,จตุรังกา
3,4,NaN
4,5,NaN
...,...,...
5603,5604,NaN
5604,5605,NaN
5605,5606,NaN
5606,5607,NaN


In [175]:
s = s.merge(submit, on='id', how='left')
s['answer'] = s['answer'].combine_first(s['Predicted']).to_frame()
s = s[['id', 'answer']]
s[s['answer'].notnull()]

,id,answer
0,1,วิเชียร ตันติพิมลพันธุ์
1,2,แมวเปอร์เซียและแมวขนสั้น
2,3,จตุรังกา
3,4,Hemidactylus
4,5,หากินเวลากลางคืน มีถิ่นกําเนิดในป่า เสือส่วนให...
...,...,...
5603,5604,3 เดือน 20 วัน
5604,5605,User Experience design (UX)
5605,5606,1. consulting firms เจ้าใหญ่และบริษัท technolo...
5606,5607,keep update อยู่เสมอ


In [178]:
s.to_csv("/content/submissQA.csv", index=False, encoding='utf_8_sig')